In [7]:
import sys
import os

# Ajouter le chemin du dossier parent au path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Maintenant tu peux importer ton config
import config

In [21]:
import pandas as pd
from openai import OpenAI
import json

In [2]:
df_players = pd.read_csv('../../data/players_stats.csv')
df_players

,Unnamed: 0,index,league,season,team,player,nation__standard,pos__standard,age__standard,born__standard,...,Performance_Crs_misc,Performance_Int_misc,Performance_TklW_misc,Performance_PKwon_misc,Performance_PKcon_misc,Performance_OG_misc,Performance_Recov_misc,Aerial Duels_Won_misc,Aerial Duels_Lost_misc,Aerial Duels_Won%_misc
0,0,0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26.0,1997.0,...,17,16,14,0,0,0,37,18,9,66.7
1,1,1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22.0,2001.0,...,117,3,15,1,0,0,70,10,20,33.3
2,2,2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28.0,1995.0,...,0,2,0,0,1,0,41,13,0,100.0
3,3,3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25.0,1999.0,...,164,25,25,0,0,0,156,37,25,59.7
4,4,4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17.0,2007.0,...,54,2,7,0,0,0,34,4,8,33.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849,2849,2849,ITA-Serie A,2425,Venezia,Nicholas Pierini,ITA,MF,25.0,1998.0,...,3,0,0,0,0,0,2,1,3,25.0
2850,2850,2850,ITA-Serie A,2425,Venezia,Nunzio Lella,ITA,MF,24.0,2000.0,...,0,0,0,0,0,0,0,0,0,NaN
2851,2851,2851,ITA-Serie A,2425,Venezia,Richie Sagrado,BEL,DF,20.0,2004.0,...,0,3,2,0,0,0,6,2,0,100.0
2852,2852,2852,ITA-Serie A,2425,Venezia,Ridgeciano Haps,SUR,DF,31.0,1993.0,...,26,7,16,0,1,0,54,14,18,43.8


In [11]:
for col in df_players.columns:
    print(col)

Unnamed: 0
index
league
season
team
player
nation__standard
pos__standard
age__standard
born__standard
Playing Time_MP_standard
Playing Time_Starts_standard
Playing Time_Min_standard
Playing Time_90s_standard
Performance_Gls_standard
Performance_Ast_standard
Performance_G+A_standard
Performance_G-PK_standard
Performance_PK_standard
Performance_PKatt_standard
Performance_CrdY_standard
Performance_CrdR_standard
Expected_xG_standard
Expected_npxG_standard
Expected_xAG_standard
Expected_npxG+xAG_standard
Progression_PrgC_standard
Progression_PrgP_standard
Progression_PrgR_standard
Per 90 Minutes_Gls_standard
Per 90 Minutes_Ast_standard
Per 90 Minutes_G+A_standard
Per 90 Minutes_G-PK_standard
Per 90 Minutes_G+A-PK_standard
Per 90 Minutes_xG_standard
Per 90 Minutes_xAG_standard
Per 90 Minutes_xG+xAG_standard
Per 90 Minutes_npxG_standard
Per 90 Minutes_npxG+xAG_standard
nation__shooting
pos__shooting
age__shooting
born__shooting
90s__shooting
Standard_Gls_shooting
Standard_Sh_shooting
Standar

In [13]:
df_players_filtered = df_players[df_players['Playing Time_90s_standard'] >= 8]
df_players_filtered.shape

(1735, 158)

In [14]:
client = OpenAI(api_key=config.Config.OPENAI_API_KEY)
player_summaries = {}

In [ ]:
for _, row in df_players_filtered.iterrows():
    player_key = f"{row['player']} ({row['team']})"
    if player_key in player_summaries:
        print(f"✅ {player_key} → Summary exist")
        continue
    
    summary_prompt = f"""
        Génère un court paragraphe décrivant le style de jeu, l'impact, les points forts et points faible du joueur suivant, basé sur ses statistiques :

        {row.to_string()}
    """
    
    response = client.chat.completions.create(
        model = config.Config.OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "Tu es un expert en football et en analyse de données statistiques."},
            {"role": "user", "content": summary_prompt}
        ],
        temperature=0.8,
        max_tokens=256
    )
    
    player_summary = response.choices[0].message.content.strip()
    player_summaries[player_key] = player_summary
    print(f"✅ {player_key} → {player_summary}")
    
    if len(player_summaries) % 50 == 0:
        with open("../../data/player_summaries.json", "w", encoding="utf-8") as f:
            json.dump(player_summaries, f, ensure_ascii=False, indent=2)
            print('📸 Snapshot')
    
player_summaries

✅ Ben White (Arsenal) → Summary exist
✅ Bukayo Saka (Arsenal) → Summary exist
✅ David Raya (Arsenal) → Summary exist
✅ Declan Rice (Arsenal) → Summary exist
✅ Ethan Nwaneri (Arsenal) → Summary exist
✅ Gabriel Magalhães (Arsenal) → Summary exist
✅ Gabriel Martinelli (Arsenal) → Summary exist
✅ Jakub Kiwior (Arsenal) → Summary exist
✅ Jurriën Timber (Arsenal) → Summary exist
✅ Kai Havertz (Arsenal) → Summary exist
✅ Leandro Trossard (Arsenal) → Summary exist
✅ Martin Ødegaard (Arsenal) → Summary exist
✅ Mikel Merino (Arsenal) → Summary exist
✅ Myles Lewis-Skelly (Arsenal) → Summary exist
✅ Riccardo Calafiori (Arsenal) → Summary exist
✅ Thomas Partey (Arsenal) → Summary exist
✅ William Saliba (Arsenal) → Summary exist
✅ Amadou Onana (Aston Villa) → Summary exist
✅ Boubacar Kamara (Aston Villa) → Summary exist
✅ Diego Carlos (Aston Villa) → Summary exist
✅ Emiliano Martínez (Aston Villa) → Summary exist
✅ Ezri Konsa (Aston Villa) → Summary exist
✅ Ian Maatsen (Aston Villa) → Summary exist


{'Ben White (Arsenal)': "Ben White, défenseur de 26 ans évoluant à Arsenal en Premier League, se distingue par son style de jeu complet et intelligent. Son impact sur le terrain est notable, bien qu'il n'ait pas marqué de buts cette saison, il a contribué avec deux passes décisives, démontrant sa capacité à participer à la construction du jeu depuis l'arrière. Ses points forts résident dans sa précision de passes, avec un taux de réussite de 83,8 %, ainsi que dans sa capacité à progresser avec le ballon, avec 394 portées réussies et 59 passes progressives. Défensivement, il est solide, avec un bon bilan d'interceptions et de tacles. Cependant, ses statistiques indiquent une certaine vulnérabilité dans les duels longs, avec un taux de réussite de seulement 48 % sur les passes longues. En somme, White est un défenseur polyvalent dont l'apport offensif et défensif fait de lui un atout précieux pour son équipe, même s'il doit encore travailler sur certains aspects de son jeu.",
 'Bukayo Sa

In [24]:
len(player_summaries)

1735

In [ ]:
with open("../../data/player_summaries.json", "w", encoding="utf-8") as f:
    json.dump(player_summaries, f, ensure_ascii=False, indent=2)